## Initialize Libs

In [6]:
#r "E:/_Projects/RateMapSeveritySaber/Ramses.Lights/bin/Release/net8.0/Ramses.Lights.dll"
#r "nuget: Microsoft.Extensions.Logging, 8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"

Installed Packages Microsoft.Extensions.Logging, 8.0.0 Microsoft.Extensions.Logging.Console, 8.0.0

In [7]:
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using System.IO;
using System.Linq;
using System.Diagnostics;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Text.Json.Serialization;
using Ramses.Lights;

var loggerFactory = LoggerFactory.Create(builder => builder.AddConsole());
var jIndented = new JsonSerializerOptions { WriteIndented = true };

## Setup Data

In [8]:
var dataset = await MapSelector.ReadMpack<List<List<LightEvent>>>();

In [9]:
var tokens = Mapping.Tokenize(dataset);

Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort
Re-Sort


In [10]:
var mark = Mapping.Train(tokens);

Processed in 16481ms
Merged in 25898ms


# Train V2

## Code with Markov 2

In [12]:
var mapFileFs = File.ReadAllBytes(@"F:\SteamLibrary\steamapps\common\Beat Saber\Beat Saber_Data\CustomWIPLevels\Laur-FairyinStrasbourg\HardStandard.dat");
var mapJson = JsonSerializer.Deserialize<JsonNode>(mapFileFs, jIndented);

var bpm = 138f;
var bps = bpm / 60f;
var spb = 1f / bps;

var noteTimes = mapJson["colorNotes"]
	.AsArray()
	.Select(n => n["b"].GetValue<float>() * spb)
	.ToArray();
var sliderBitsTimes = mapJson["burstSliders"]
	.AsArray()
	.SelectMany(n => {
		var start = n["b"].AsValue().GetValue<float>() * spb;
		var end = n["tb"].AsValue().GetValue<float>() * spb;
		var duration = end - start;
		var bitTime = duration / n["sc"].GetValue<int>();
		return Enumerable.Range(0, n["sc"].GetValue<int>()).Select(i => start + i * bitTime);
	})
	.ToArray();

var allTimes = noteTimes
	.Concat(sliderBitsTimes)
	.Select(n => float.Round(n, 3))
	.Distinct()
	.OrderBy(t => t)
	.ToArray();

record GenElem(
	float Time,
	LightToken Token,
	bool UseChain
);

List<GenElem> genList = [];
List<LightToken> combo = new();

byte stateRotLeft = 0; // ET 2
byte stateRotRight = 0; // ET 3

for (int i = 0; i < allTimes.Length - 1; i++)
{
	var time = allTimes[i];

	combo.Clear();
	for(int j = 0; j < 16; j++)
	{
		var next = mark.Chain(genList.Where(x => x.UseChain).Take(^2..).Select(x => x.Token)).FirstOrDefault();
		if(combo.Any(x => x.Type == next.Type))
		{
			continue;
		}
		combo.Add(next);
		if (next == null || next.Combo)
		{
			break;
		}
	}
	
	foreach (var next in combo)
	{
		if (next.Type == EventType.LEFT && next.StateValue != stateRotLeft)
		{
			genList.Add(new(time, new LightToken(EventType.LEFT_ROT, 0, stateRotLeft, 0, next.FloatValue, false), false));
			stateRotLeft = next.StateValue;
		}
		else if (next.Type == EventType.RIGHT && next.StateValue != stateRotRight)
		{
			genList.Add(new(time, new LightToken(EventType.RIGHT_ROT, 0, stateRotRight, 0, next.FloatValue, false), false));
			stateRotRight = next.StateValue;
		}

		var t = LightToken.CategorizeLightSpeed(allTimes[i + 1] - allTimes[i]);
		genList.Add(new (time, new LightToken(next.Type, t, next.Value, 0, next.FloatValue, false), true));
	}
}

mapJson["basicBeatmapEvents"] = new JsonArray(genList.Select(t => {
	var j = new JsonObject();
	j["b"] = float.Round(t.Time * bps, 3);
	j["et"] = t.Token.Type;
	j["i"] = t.Token.Value;
	j["f"] = 1f;
	return j;
}).ToArray());

var newMap = JsonSerializer.Serialize(mapJson, jIndented);
File.WriteAllText(@"F:\SteamLibrary\steamapps\common\Beat Saber\Beat Saber_Data\CustomWIPLevels\Laur-FairyinStrasbourg\NormalStandard.dat", newMap);